# DEDS-week 7, Hoorcollege 1 - Missende waarden in categorische data

## Libraries importeren

In [ ]:
import pandas as pd
import pyodbc
from ydata_profiling import ProfileReport

We importeren een extra library, die we vorige week nog niet gebruikten: ydata_profiling, waaruit we ProfileReport halen. Waar deze voor is, komt verderop aan bod.

## DB-connecties maken

In [ ]:
sales_conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=..\GO-databases DEDS-week 4 & 5\go_sales.accdb;')

## Dataframe aanmaken vanuit SELECT op databasetabel

In [ ]:
product = pd.read_sql("SELECT * FROM product", sales_conn)
product

## Kwaliteitsoverzicht

In [ ]:
profile = ProfileReport(product, title = 'Kwaliteitsrapportage product')
# profile.to_widgets()

Met het geïmporteerde rapport kunnen we een statistisch overzicht maken van een aantal kengetallen en correlaties in deze dataset. Enige nadeel: het duurt vrij lang voordat dit standaardrapport klaar is. Met name het tabblad "Alerts" onder "Overview" is interessant: die doet namelijk onder andere uitspraken over de datakwaliteit en missende waarden mogen daarin uiteraard niet ontbreken. Klik er voor de aardigheid gewoon eens doorheen en dan zie je wat Profiling allemaal voor je uit kan rekenen. Het is altijd goed om dit soort statistieken inzichtelijk te hebben voor snel inzicht in je ingeladen dataset, alvorens je daar mutaties op gaat toepassen.

## Missende waarden afhandelen

### Mogelijke methoden

Binnen de Data Science wordt er onderscheid gemaakt tussen 2 soorten data:
- Categorische data: dit zijn datapunten waarop alleen een telberekening uitgevoerd kan worden.
- Continue data: dit zijn getallen waarop verschillende berekeningen uitgevoerd kunnen worden (som, gemiddelde, mediaan, minimum, maximum, enzovoort). Hoe je in dit soort kolommen  missende data handelt komt woensdag aanbod.

Hoe je missende waarden gaat invullen hangt heel sterk af van de soort data die in de betreffende kolom staat.
3 mogelijkheden voor het invullen van missende categorische datawaarden:
- Waarden op 'onbekend' zetten. Gebruik deze methode als...
    - De kolom allemaal verschillende waarden bevat, die niet afgeleid kunnen worden van waarden uit andere kolommen.
    - Je op korte termijn niet geïnteresseerd bent in de analysemogelijkheden van deze kolom.
- De waarde pakken die het vaakst voorkomt. Deze waarde wordt ook wel de modus genoemd. Gebruik deze methode als verreweg de meeste kolomvelden dezelfde waarde bevatten. De spreiding van waarden in je kolom is dan dus niet heel groot. 
- De 'buur-analyse'. In tegenstelling tot de andere twee methoden analyseer je hierbij het héle dataframe, in plaats van alleen de betreffende kolom met de missende waarden  Deze methode bestaat uit 2 stappen...
    - Je zoekt naar overeenkomstige eigenschappen tussen rijen waarin de betreffende kolom niet is ingevuld.
    - Je zoekt in de overkoepelende tabel naar rijen met die eigenschappen en probeert de juiste vulwaarden te achterhalen.

In de onderstaande coderegels wordt per categorische kolom uit de producttabel achterhaald wat de juiste vulmethode is en wordt gedemonstreerd hoe je deze toepast in Python.

### INTRODUCTION_DATE

De kolom INTRODUCTION_DATE bestaat alleen maar uit categorische data: er staan immers alleen datums in en geen getallen. Hoeveel missende waarden heeft deze kolom ook alweer? Laten we nog eens kijken...

In [ ]:
product['INTRODUCTION_DATE'].isna().sum()

Er zijn dus 5 waarden die nu nog missen. Laten we met de eerste methode oefenen: de 'onbekend'-methode. We besluiten namelijk ter plekke dat we niet geïnteresseerd zijn in het analyseren van introductiedata en dit is een geldig argument om deze methode te hanteren.

In [ ]:
product['INTRODUCTION_DATE'] = product['INTRODUCTION_DATE'].fillna('onbekend')
product['INTRODUCTION_DATE'].isna().sum()

Met de eerste coderegel vullen we de waarde 'onbekend' in met de .fillna(...)-methode. Met de tweede coderegel vragen we vervolgens nog een keer het aantal missende waarden op. We verwachten nu uiteraard dat deze 0 is en dat klopt ook! Operatie geslaagd dus!

Maar wacht... eigenlijk hebben we nu toch geen waarde toegevoegd? Eerst miste er een regel en nu staat daar simpelweg 'onbekend', wat is hier het nut van? 2 redenen...
- Als je Machine-Learningmodellen gaat trainen mag de data die je erin stopt helemaal geen missende waarden bevatten, anders crasht het model.
- Je ziet in reporting-programma's, bijvoorbeeld Power BI & Qlikview, geen irritante 'blanks' meer staan. Je weet dus dat je op zijn minst heb nagedacht over hoe je missende waarden afhandelt.

### LANGUAGE

Ook de kolom LANGUAGE bestaat alleen maar uit categorische data: er staat immers alleen tekst in en geen getallen. Hoeveel missende waarden hebben we hier ook alweer? Let's find out!

In [ ]:
product['LANGUAGE'].isna().sum()

We hebben vorige week al gezien dat bijna elke waarde in LANGUAGE 'EN' is. De categorische data in deze kolom is dus niet heel erg verdeeld, wat een goed argument is om de 6 ontbrekende waarden alsnog in te vullen met de 'modus'-methode. Uiteraard dienen we daarvoor eerst de modus te bepalen, voordat we .fillna(...) kunnen vullen. Dat doen we met onderstaande code:

In [ ]:
modus = product['LANGUAGE'].mode()[0]
modus

We zien dat de waarde 'EN' inderdaad het vaakst voorkomt in LANGUAGE. We slaan deze op in de variabele 'modus'.

In [ ]:
product['LANGUAGE'] = product['LANGUAGE'].fillna(modus)
product['LANGUAGE'].isna().sum()

We gebruiken ten slotte de modus om de .fillna() in te vullen. Ook hier zien we dat de operatie is gelukt :-)

### PRODUCT_TYPE_CODE

En... wéér hebben we categorische data. In PRODUCT_TYPE_CODE staan weliswaar getallen, maar zij geven categorieën aan. De som en/of het gemiddelde van dergelijke data berekenen zou heel erg onnuttig zijn.

In [ ]:
product['PRODUCT_TYPE_CODE'].isna().sum()

6 missende rijen, mooi! Dan kunnen we aan de slag met de derde invulmethode voor categorische data: de 'buur'-analyse. De eerste stap omvat het maken van een subset van alle kolommen van rijen waar PRODUCT_TYPE_CODE leeg is om te kijken welke andere kolommen wél ingevuld zijn.

In [ ]:
product.loc[product['PRODUCT_TYPE_CODE'].isna(), :]

In deze subset gaan we vervolgens op zoek naar gelijkenissen tussen rijen.

#### PRODUCT_NAME bevat 'Granite'

Het valt ons ten eerste op dat in 'PRODUCT_NAME' twee keer het woord 'Granite' voorkomt. Als we in het overkoepelende dataframe zoeken op rijen met productnamen die dit woord bevatten kunnen we misschien achterhalen welk producttype hier het vaakst bijhoort.

In [ ]:
subset_product_name_contains_granite = product.loc[(~product['PRODUCT_NAME'].isna()) & (product['PRODUCT_NAME'].str.contains('Granite')), :]
subset_product_name_contains_granite

We zien hier dat producten die 'Granite' in hun naam hebben verreweg het vaakst bij producttype 9 horen. Vanaf hier kunnen we de aanname doen dat we missende waarden met productnamen met 'Granite' erin met 9 mogen invullen.

In [ ]:
product.loc[(~product['PRODUCT_NAME'].isna()) & (product['PRODUCT_NAME'].str.contains('Granite')), 'PRODUCT_TYPE_CODE'] = subset_product_name_contains_granite['PRODUCT_TYPE_CODE'].fillna(9)
product['PRODUCT_TYPE_CODE'].isna().sum()

Waarom is de code aan de linkerkant van '=' zo lang? Omdat we een gedeelte van het originele dataframe ('product') willen overschrijven. Gelukkig kunnen we aan de rechterkant van '=' de gemaakte subset gebruiken. Van de 6 rijen waren er 2 die 'Granite' bevatten en daarvan is de producttypecode inmiddels ingevuld. Er zouden dus nog 4 lege waarden over moeten blijven en dat klopt!

#### INTRODUCTION_DATE bevat jaar 2011 en 2019

In [ ]:
product.loc[product['PRODUCT_TYPE_CODE'].isna(), :]

Ten tweede zien we dat bij de ene 2 rijen waar de producttypecode mist, het jaartal 2011 is, bij de andere 2 rijen is dit 2019. Zou dat een goede tweede aanwijzing kunnen zijn? Let's find out! Hiervoor is het handig om eerst even een extra kolom 'INTRODUCTION_YEAR' te maken.

In [ ]:
product['INTRODUCTION_YEAR'] = product['INTRODUCTION_DATE'].str[-4:]
product

Met bovenstaande code lees je de laatste 4 karakters uit van elke string die in de kolom 'INTRODUCTION_DATE' staat. Met zijn viertjes geven zij het jaartal weer, dat vervolgens wordt geëxporteerd naar een nieuwe kolom: 'INTRODUCTION_YEAR'.

We zoeken weer in het overkoepelende dataframe op rijen met dezelfde eigenschappen (in dit geval: introductiejaar = 2011 OF 2019). Zo hopen we wederom te achterhalen welk producttype hier het vaakst bijhoort.

In [ ]:
subset_introduction_year_2011_2019 = product.loc[(product['INTRODUCTION_YEAR'] == '2011') | (product['INTRODUCTION_YEAR'] == '2019'), :]
subset_introduction_year_2011_2019

Hmm... nogal onoverzichtelijk he. Gelukkig kunnen we, net als in SQL, groeperen op introductiejaar en producttypecode, om vervolgens met count() het aantal rijen te tellen. 'PRODUCT_NUMBER' kan misschien verwarrend zijn, je moet namelijk jammer genoeg per se een kolom uitkiezen waarvan de rijen geteld worden. Om deze verwarring weg te nemen wijzigen we maar snel deze kolomnaam naar 'Aantal producten'.

In [ ]:
grouped_subset_introduction_year_2011_2019 = subset_introduction_year_2011_2019.groupby(['INTRODUCTION_YEAR', 'PRODUCT_TYPE_CODE'])['PRODUCT_NUMBER'].count().reset_index()
grouped_subset_introduction_year_2011_2019 = grouped_subset_introduction_year_2011_2019.rename(columns = {'PRODUCT_NUMBER' : 'Aantal producten'})
grouped_subset_introduction_year_2011_2019.sort_values(['INTRODUCTION_YEAR', 'Aantal producten'], ascending = [True, False])

We zien dat in jaartal 2011 producttypecode 1 het vaakst voorkomt, in jaartal 2019 is dat producttypecode 19. Hiermee kunnen we de lege waarden vullen.

In [ ]:
product.loc[(product['INTRODUCTION_YEAR'] == '2011'), 'PRODUCT_TYPE_CODE'] = product.loc[(product['INTRODUCTION_YEAR'] == '2011'), 'PRODUCT_TYPE_CODE'].fillna(1)
product.loc[(product['INTRODUCTION_YEAR'] == '2019'), 'PRODUCT_TYPE_CODE'] = product.loc[(product['INTRODUCTION_YEAR'] == '2019'), 'PRODUCT_TYPE_CODE'].fillna(19)
product['PRODUCT_TYPE_CODE'].isna().sum()

Wederom willen we een gedeelte van het originele dataframe ('product') overschrijven (linkerkant '='). Het aantal lege waarden in PRODUCT_TYPE zou nu 0 moeten zijn en dat klopt! Yay for us :-D